In [1]:
import spotify_info
import pandas as pd
from tqdm import tqdm
import simplejson as json
import asyncio
import numpy as np
import pickle

In [2]:
spotify_info.test()

Testing


In [3]:
track_uri = "spotify:track:0UaMYEvWZi0ZqiDOoHU3YI"
artist_uri = "spotify:artist:2wIVse2owClT7go1WT98tk"
album_uri = "spotify:album:6vV5UrXcfyQD1wu4Qo2I9K"

## Read all files

In [4]:
files = ['mpd.slice.0-999.json'
,'mpd.slice.1000-1999.json'
,'mpd.slice.2000-2999.json'
,'mpd.slice.3000-3999.json'
,'mpd.slice.4000-4999.json'
,'mpd.slice.5000-5999.json'
,'mpd.slice.6000-6999.json'
,'mpd.slice.7000-7999.json'
,'mpd.slice.8000-8999.json'
,'mpd.slice.9000-9999.json']

## collect all unique uri

In [5]:
def get_all_tracks_list(filename):
    """Extract tracks from the playlists using a JSON data source file

    Args:
        filename (str): The file path of json file

    Returns:
        list: A list of dataframe. Contains tracks information from playlist, each attribute in the list is one collection of tracks of one playlist
    """
    with open(filename,'r') as f:
        data = json.load(f)

    ls_df_tracks = []
    for playlist in data['playlists']:
        df_temp = pd.DataFrame.from_records(playlist['tracks'])
        df_temp['pid'] = playlist['pid']
        ls_df_tracks.append(df_temp)

    return ls_df_tracks


In [6]:
ls_df_tracks = []
for f_name in tqdm(files):
    ls_df_tracks += get_all_tracks_list("../data/"+f_name)

100%|██████████| 10/10 [00:08<00:00,  1.14it/s]


In [7]:
df_tracks = pd.concat(ls_df_tracks, ignore_index=True)
df_tracks

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,pid
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,0
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,0
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,0
...,...,...,...,...,...,...,...,...,...
664707,50,Keith Urban,spotify:track:6ZOPiKQeibCn7fP8dncucL,spotify:artist:0u2FHSq3ln94y5Q57xazwf,Blue Ain't Your Color,spotify:album:1r7ABqzNXQnUPAH3ZjrHMn,230600,Ripcord,9999
664708,51,Dustin Lynch,spotify:track:7pxhKtuTwofDIdgHx2DcVK,spotify:artist:1dID9zgn0OV0Y8ud7Mh2tS,Seein' Red,spotify:album:23cuZhPWDfX1uKD4qwuv7t,194893,Current Mood,9999
664709,52,Sam Hunt,spotify:track:7mldq42yDuxiUNn08nvzHO,spotify:artist:2kucQ9jQwuD8jWdtR9Ef38,Body Like A Back Road,spotify:album:2N7kidh1wA9EoLdf16QWrz,165386,Body Like A Back Road,9999
664710,53,Little Big Town,spotify:track:23TxRN09aR1RB0G0tFoT0b,spotify:artist:3CygdxquGHurS7f9LjNLkv,Better Man,spotify:album:2aQOzEjLzPkffXDwREXdAh,263120,The Breaker,9999


In [8]:
# unique values of the columns
print("track_uri",len(df_tracks.track_uri.unique()))
print("artist_uri",len(df_tracks.artist_uri.unique()))
print("album_uri",len(df_tracks.album_uri.unique()))


track_uri 170089
artist_uri 35892
album_uri 81565


## TODO using the spotify api to collect infomation

In [9]:
async def get_all_tracks(list_df):
    ls_all_info_df = []
    for i,df in enumerate(list_df):
        print('track',i)
        ls_info = []
        for uri in tqdm(df):
            result = await spotify_info.get_track_info(uri)
            ls_info.append(result)


        result_df = pd.DataFrame.from_records(ls_info)
        result_df.to_pickle(f"../data/track_info_{i}.pkl") 
        ls_all_info_df.append(result_df)
    df_all_info = pd.concat(ls_all_info_df, ignore_index=True)
    df_all_info.to_pickle(f"../data/track_info_all.pkl") 

async def get_all_albums(list_df):
    ls_all_info_df = []
    for i,df in enumerate(list_df):
        print('albums',i)
        ls_info = []
        for uri in tqdm(df):
            result = await spotify_info.get_album_info(uri)
            ls_info.append(result)


        result_df = pd.DataFrame.from_records(ls_info)
        result_df.to_pickle(f"../data/album_info_{i}.pkl") 
        ls_all_info_df.append(result_df)
    df_all_info = pd.concat(ls_all_info_df, ignore_index=True)
    df_all_info.to_pickle(f"../data/album_info_all.pkl") 

async def get_all_artists(list_df):
    ls_all_info_df = []
    for i,df in enumerate(list_df):
        print('artists',i)
        ls_info = []
        for uri in tqdm(df):
            result = await spotify_info.get_artist_info(uri)
            ls_info.append(result)


        result_df = pd.DataFrame.from_records(ls_info)
        result_df.to_pickle(f"../data/artist_info_{i}.pkl") 
        ls_all_info_df.append(result_df)
    df_all_info = pd.concat(ls_all_info_df, ignore_index=True)
    df_all_info.to_pickle(f"../data/artist_info_all.pkl") 

In [10]:
list_df_track = np.array_split(df_tracks.track_uri.unique(), 17)
list_df_artist = np.array_split(df_tracks.artist_uri.unique(), 4)
list_df_album = np.array_split(df_tracks.album_uri.unique(), 8)


In [11]:
# list_df_track = np.array_split(df_tracks.track_uri.unique(), 1700)
# list_df_artist = np.array_split(df_tracks.artist_uri.unique(), 400)
# list_df_album = np.array_split(df_tracks.album_uri.unique(), 800)


In [12]:

# await get_all_tracks(list_df_track)
await get_all_artists(list_df_artist)
await get_all_albums(list_df_track)

# await asyncio.gather(
#     get_all_tracks(list_df_track),
#     get_all_artists(list_df_artist),
#     get_all_albums(list_df_track)
# )
# async def main():
#     await asyncio.gather(
#         get_all_tracks(list_df_track[:2]),
#         get_all_artists(list_df_artist[:2]),
#         get_all_albums(list_df_track[:2])
#     )


artists 0


  0%|          | 2/8973 [3:14:50<14566:09:21, 5845.30s/it]


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/artists/6vWDO969PvNqNYHIOW5v0m:
 The access token expired, reason: None

: 

## TODO merge the collected information back to track, artist, album dataset